In [ ]:
!pip install xgboost imbalanced-learn --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
import pandas as pd

In [18]:
df=pd.read_csv("Fraud.csv")

In [19]:
print(df.head())
print(df.info())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0      0.0             0.0  
1  M2044282225             0.0             0.0      0.0             0.0  
2   C553264065             0.0             0.0      1.0             0.0  
3    C38997010         21182.0             0.0      1.0             0.0  
4  M1230701703             0.0             0.0      0.0             0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28297 entries, 0 to 28296
Data columns (total 11 column

In [20]:
features=['type','amount','oldbalanceOrg','newbalanceOrig', 'oldbalanceDest','newbalanceDest']
target='isFraud'

In [23]:
df=df.dropna(subset=['isFraud'])

In [24]:
X=df.drop(columns=['isFraud','nameOrig','nameDest'])
y=df['isFraud']

In [21]:
X=df[features]
y=df[target]

In [22]:
X=pd.get_dummies(X,columns=['type'],drop_first=True)

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [26]:
print("Traing data shape:",X_train.shape)
print("Test data shape:",X_test.shape)


Traing data shape: (22636, 8)
Test data shape: (5660, 8)


In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
labelencoder=LabelEncoder()
for col in X_train.columns:
    if X_train[col].dtype=='object':
        X_train[col]=labelencoder.fit_transform(X_train[col])
        X_test[col]=labelencoder.transform(X_test[col])


In [32]:
from imblearn.over_sampling import SMOTE

In [33]:
smote=SMOTE(random_state=42)
X_train_resampled,y_train_resampled=smote.fit_resample(X_train,y_train)

In [35]:
print("after SMOTE -Training data shape:",X_train_resampled.shape)
print("After SMOTE-Traing target distributuion:\n",y_train_resampled.value_counts())

after SMOTE -Training data shape: (45138, 8)
After SMOTE-Traing target distributuion:
 isFraud
0.0    22569
1.0    22569
Name: count, dtype: int64


In [36]:
X_train=X_train[~y_train.isna()]
y_train=y_train.dropna()

In [37]:
X_test=X_test[~y_test.isna()]
y_test=y_test.dropna()

In [38]:
print("NaNs in Y_train:",y_train.isna().sum())
print("NaNs in Y_test:",y_test.isna().sum())

NaNs in Y_train: 0
NaNs in Y_test: 0


In [39]:
xgb=XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    scale_pos_weight=(y_train.value_counts()[0]/y_train.value_counts()[1])


)

In [40]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [41]:
y_pred=xgb.predict(X_test)
y_prob=xgb.predict_proba(X_test)[:,1]

In [42]:
mask=~np.isnan(y_test)
y_test_clean=y_test[mask]
y_pred_clean=y_pred[mask]
y_prob_clean=y_prob[mask]

In [43]:
print("Confusion Matrix:\n",confusion_matrix(y_test_clean,y_pred_clean))
print("Classification Report:\n",classification_report(y_test_clean,y_pred_clean))
print("Accuracy Score:",roc_auc_score(y_test_clean,y_prob_clean))

Confusion Matrix:
 [[5641    2]
 [   4   13]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5643
         1.0       0.87      0.76      0.81        17

    accuracy                           1.00      5660
   macro avg       0.93      0.88      0.91      5660
weighted avg       1.00      1.00      1.00      5660

Accuracy Score: 0.9830086207795186
